- Факторы которые влияют на энергопотребление? - почему?
- найти факторы и сделать рейтинг факторов
- Один дом, один файл.
- PDF и код юпитера
- К репозиторию файл README


### Инициализация и загрузка данных

In [48]:
import pandas as pd

# Относительный путь к папке где лежать файлы
smart_meters_folder_path = 'smart-meters-in-london/'

# Получение полного пути файла
file_acorn_details = '{}acorn_details.csv'.format(smart_meters_folder_path)
file_uk_bank_holidays = '{}uk_bank_holidays.csv'.format(smart_meters_folder_path)
file_weather_daily_darksky = '{}weather_daily_darksky.csv'.format(smart_meters_folder_path)
file_weather_hourly_darksky = '{}weather_hourly_darksky.csv'.format(smart_meters_folder_path)
file_informations_households = '{}informations_households.csv'.format(smart_meters_folder_path)
file_daily_dataset = '{}daily_dataset.csv/daily_dataset.csv'.format(smart_meters_folder_path)

# Загрузка файлов в Pandas
df_acorn_details = pd.read_csv(file_acorn_details, encoding='ANSI')
df_uk_bank_holidays = pd.read_csv(file_uk_bank_holidays, encoding='ANSI')
df_weather_daily_darksky = pd.read_csv(file_weather_daily_darksky, encoding='ANSI')
df_weather_hourly_darksky = pd.read_csv(file_weather_hourly_darksky, encoding='ANSI')
df_informations_households = pd.read_csv(file_informations_households, encoding='ANSI')
df_daily_dataset = pd.read_csv(file_daily_dataset, encoding='ANSI')

In [10]:
df_daily_dataset.head()

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066


In [80]:
df_informations_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0


### Тип счетчика в зависимости от класса ACORN
Хочу получить процентное соотношение ToU/Std в зависимости от класса

In [49]:
altered_df_informations_households = df_informations_households.loc[:][:]

In [50]:
def count_ToU(row):
    if row['stdorToU'] == 'ToU':
        return 1
    return 0

def count_Std(row):
    if row['stdorToU'] == 'Std':
        return 1
    return 0

def ToU_percentage(row):
    return row['ToU'] / (row['ToU'] + row['Std'])

In [51]:
altered_df_informations_households['ToU'] = df_informations_households.apply(count_ToU, axis=1)
altered_df_informations_households['Std'] = df_informations_households.apply(count_Std, axis=1)

In [53]:
altered_df_informations_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file,ToU,Std
0,MAC005492,ToU,ACORN-,ACORN-,block_0,1,0
1,MAC001074,ToU,ACORN-,ACORN-,block_0,1,0
2,MAC000002,Std,ACORN-A,Affluent,block_0,0,1
3,MAC003613,Std,ACORN-A,Affluent,block_0,0,1
4,MAC003597,Std,ACORN-A,Affluent,block_0,0,1


In [78]:
counter_type_stats = altered_df_informations_households.groupby(by = 'Acorn')['ToU', 'Std'].sum()
counter_type_stats['ToU_perc'] = counter_type_stats.apply(ToU_percentage, axis=1)

In [88]:
counter_type_stats['ToU_perc']

Acorn
ACORN-     1.000000
ACORN-A    0.248408
ACORN-B    0.160000
ACORN-C    0.231788
ACORN-D    0.250000
ACORN-E    0.216337
ACORN-F    0.223684
ACORN-G    0.229268
ACORN-H    0.184615
ACORN-I    0.156863
ACORN-J    0.276786
ACORN-K    0.133333
ACORN-L    0.222222
ACORN-M    0.150442
ACORN-N    0.171053
ACORN-O    0.155340
ACORN-P    0.200000
ACORN-Q    0.143201
ACORN-U    0.204082
Name: ToU_perc, dtype: float64

In [74]:
altered_df_informations_households.groupby(by = 'Acorn')['ToU', 'Std'].sum()

,ToU,Std
Acorn,,
ACORN-,2,0
ACORN-A,39,118
ACORN-B,4,21
ACORN-C,35,116
ACORN-D,73,219
ACORN-E,339,1228
ACORN-F,153,531
ACORN-G,47,158
ACORN-H,84,371


### Расчет среднего потребления энергии в зависимости от класса

In [83]:
daily_dataset_with_hh_info = df_daily_dataset.merge(df_informations_households, how = 'left', on = 'LCLid')

In [87]:
daily_dataset_with_hh_info[ daily_dataset_with_hh_info['energy_count'] == 48].head()

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,file
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031,Std,ACORN-E,Affluent,block_22
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064,Std,ACORN-E,Affluent,block_22
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065,Std,ACORN-E,Affluent,block_22
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066,Std,ACORN-E,Affluent,block_22
5,MAC000131,2011-12-20,0.2180,0.357500,1.077,48,0.287597,17.160,0.066,Std,ACORN-E,Affluent,block_22
